In [73]:
import cvxpy as cp
import numpy as np
import random
import time

In [107]:
def get_date(n = 10,m = 3,sumc = 9):
    value = cp.Parameter(shape=(n,m))
    capacity = cp.Parameter(m)
    #value.value = np.random.rand(n,m)
    value.value = np.random.randint(1,9,size=(n,m))
    capacity.value = np.random.randint(1,sumc/m,size=m)
    return value, capacity

value, capacity = get_date()
#print(value.value)
#print(capacity.value)

In [108]:
%%time
def Random(value,capacity):
    value,capacity = value.value,capacity.value
    n, m = len(value), len(capacity)
    sumc = sum(capacity)
    rand = random.sample(range(1,n),sumc)
    assign = [0 for i in range(m)]
    ans = 0
    result = [0 for i in range(sumc)]
    for i in range(sumc):
        tmp = 0
        for j in range(m):
            if value[rand[i]][j] > tmp and assign[j]<capacity[j]:
                result[i],tmp = j,value[rand[i]][j]
        assign[result[i]] += 1
        ans += tmp
    print(ans)
    return ans,rand[:sumc],result
    
print(Random(value,capacity))

23
(23, [3, 2, 8, 7], [0, 2, 2, 1])
CPU times: user 607 µs, sys: 0 ns, total: 607 µs
Wall time: 413 µs


In [109]:
%%time
def DP(value,capacity):
    value,capacity = value.value,capacity.value
    n, m = len(value), len(capacity)
    mi = [1]
    for i in range(1,m+1):
        mi.append(mi[i-1]*(capacity[m-i]+1))
    f = np.zeros(mi[m])
    for i in range(n):
        for j in range(mi[m]-1,-1,-1):   
            for k in range(m):
                if ((j%mi[m-k])//mi[m-k-1]>0):        
                    f[j] = max(f[j],f[j-mi[m-k-1]]+value[i,k])
    return f[mi[m]-1]

print(DP(value,capacity))

30.0
CPU times: user 601 µs, sys: 47 µs, total: 648 µs
Wall time: 474 µs


In [110]:
%%time

def Opt(value,capacity):
    n, m = len(value.value), len(capacity.value)
    x = cp.Variable(shape=(n,m),boolean=True)
    object = cp.Maximize(cp.sum(cp.multiply(value,x)))
    constrains = [cp.sum(x,0)<=capacity,cp.sum(x,1)<=1]
    prob = cp.Problem(object,constrains)
    prob.solve('GLPK_MI',verbose=True) # ECOS_BB CBC
    if prob.status == 'optimal':
        print(prob.value)
        return x.value,prob.value

print(Opt(value,capacity))



30.0
(array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]), 30.0)
CPU times: user 3.74 ms, sys: 10.4 ms, total: 14.2 ms
Wall time: 9.96 ms


In [113]:
%%time

def Opt_dual(value,capacity):
    n, m = len(value.value), len(capacity.value)
    lamb = cp.Variable(n,nonneg=True)
    nu = cp.Variable(m,nonneg=True)
    object = cp.Minimize(cp.sum(lamb)+cp.sum(cp.multiply(capacity,nu)))
    constrains = [cp.vstack([cp.hstack([lamb[i]+nu[j] for j in range(m)]) for i in range(n)]) >= value]
    prob = cp.Problem(object,constrains)
    prob.solve('GLPK_MI',verbose=True) # ECOS_BB CBC
    if prob.status == 'optimal':
        return constrains[0].dual_value,prob.value

print(Opt_dual(value,capacity))

(None, 30.0)
CPU times: user 27.4 ms, sys: 420 µs, total: 27.8 ms
Wall time: 23.9 ms


In [103]:
def get_time(f,value,capacity):
    start = time.perf_counter()
    f(value,capacity)
    end = time.perf_counter()
    return end-start

def get_times(n = 10,m = 3, sumc = 9):
    t = []
    value, capacity = get_date(n,m,sumc)
    t.append(get_time(Random,value,capacity))
    #t.append(get_time(DP,value,capacity))
    t.append(get_time(Opt,value,capacity))
    #t.append(get_time(Opt_dual,value,capacity))
    return t

In [104]:
parm = [[10,3,9],[100,3,30],[1000,3,300],[100,7,70],[1000,7,700]]
for i in parm:
    print(get_times(i[0],i[1],i[2]))


26
31.0
[0.0002103000006172806, 0.008740600002056453]
78
103.0
[0.0006311999968602322, 0.018989299998793285]
489
672.0
[0.00033460000122431666, 0.4784049000008963]
323
367.0
[0.0004371999966679141, 0.04229180000402266]
2693
3056.0
[0.0012257000053068623, 2.592032399996242]


In [96]:
for i in parm[0:-2]:
    value, capacity = get_date(i[0],i[1],i[2])
    print(get_time(DP,value,capacity))

0.0006403999977919739
0.13651499999832595
22.497699999999895


In [ ]:
%%time
import cvxpy as cp
import numpy as np

n = 10
m = 3
value = cp.Parameter(n*m)
capacity = cp.Parameter(n+m)
A = cp.Parameter(shape=(n+m,m*n))
value.value = np.random.rand(n*m)
capacity.value = np.random.randint(1,3,size=m+n)
for i in range(n+m):
    if i<n:
        capacity.value[i] = 1
    else:
        capacity.value[i] = np.random.randint(1,3)
A.value = np.zeros((n+m,n*m))
for i in range(n):
    for j in range(m):
        A.value[i][i*m+j] = 1
for i in range(n,m+n):
    for j in range(n):
        A.value[i][j*m+i-n] = 1

In [ ]:
%%time
x = cp.Variable(shape=(n*m),nonneg=True)
object = cp.Minimize(cp.sum(cp.multiply(-value,x)))
constrains = [-A@x>=-capacity]
prob = cp.Problem(object,constrains)
prob.solve()
print(prob.status)
print(prob.value)

In [ ]:
%%time
lamb = cp.Variable(shape=(m+n),nonneg=True)
object = cp.Minimize(cp.sum(cp.multiply(capacity,lamb)))
constrains = [A.T@lamb>=value]
prob = cp.Problem(object,constrains)
prob.solve()
print(prob.status)
print(prob.value)

In [ ]:
%%time

assert m==3,"m != 3"

f = np.zeros(tuple(capacity.value+1))
for i in range(n):
    for j in range(capacity.value[0],-1,-1):
        for k in range(capacity.value[1],-1,-1):
            for l in range(capacity.value[2],-1,-1):
                if (j):
                    f[j][k][l] = max(f[j][k][l],f[j-1][k][l]+value.value[i,0])
                if (k):
                    f[j][k][l] = max(f[j][k][l],f[j][k-1][l]+value.value[i,1])
                if (l):
                    f[j][k][l] = max(f[j][k][l],f[j][k][l-1]+value.value[i,2])

print(f[capacity.value[0]][capacity.value[1]][capacity.value[2]])
#print(f)

In [ ]:
%%time

assert m==3,"m != 3"

f = np.zeros((n+1,)+tuple(capacity.value+1))
for i in range(1,n+1):
    for j in range(0,capacity.value[0]+1,1):
        for k in range(0,capacity.value[1]+1,1):
            for l in range(0,capacity.value[2]+1,1):
                f[i][j][k][l] = f[i-1][j][k][l]
                if (j):
                    f[i][j][k][l] = max(f[i][j][k][l],f[i-1][j-1][k][l]+value.value[i-1,0])
                if (k):
                    f[i][j][k][l] = max(f[i][j][k][l],f[i-1][j][k-1][l]+value.value[i-1,1])
                if (l):
                    f[i][j][k][l] = max(f[i][j][k][l],f[i-1][j][k][l-1]+value.value[i-1,2])

print(f[n][capacity.value[0]][capacity.value[1]][capacity.value[2]])